In [ ]:
import chipwhisperer as cw
from chipwhisperer.capture.targets.CW305 import CW305
scope = cw.scope()
target = CW305()

In [ ]:
project = cw.createProject("projects/Tutorial_HW_CW305.cwp", overwrite=True)
tc = project.newSegment()

In [ ]:
# setup scope parameters
scope.gain.gain = 30
scope.gain.mode = "high"
scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "extclk_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "disabled"

In [ ]:
# program target FPGA bitfile
# pick the correct bitfile for your CW305 board:
#bitstream = r" ../../hardware/victims/cw305_artixtarget/fpga/vivado_examples/aes128_verilog/aes128_verilog.runs/impl_35t/cw305_top.bit"
bitstream = r"../../hardware/victims/cw305_artixtarget/fpga/vivado_examples/aes128_verilog/aes128_verilog.runs/impl_100t/cw305_top.bit"
target.con(bsfile=bitstream)

In [ ]:
# setup FPGA:
target.vccint_set(1.0)
# we only need PLL1 (board jumper settings: J16=0, K16=1, K15=1, L14=1)
target.pll.pll_enable_set
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(True, 1)
target.pll.pll_outenable_set(False, 2)

# run at 10 MHz:
target.pll.pll_outfreq_set(10E6, 1)

# 1ms is plenty of idling time
target.clkusbautooff = True
target.clksleeptime = 1

# ensure ADC is locked:
scope.clock.resetAdc()
assert (scope.clock.adc_locked), "ADC failed to lock"

In [ ]:
#Capture Traces
from tqdm import tqdm
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

traces = []
textin = []
keys = []
N = 5000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    textout = target.readOutput()  # read encryption result (required by attack!)
    trace = scope.getLastTrace()
    traces.append(trace)
    tc.addTrace(trace, text, textout, key)

In [ ]:
# plot a trace
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width=800)

xrange = range(len(traces[0]))
p.line(xrange, traces[2], line_color="red")
show(p)

In [ ]:
# save project and disconnect
project.appendSegment(tc)
project.save()
scope.dis()
target.dis()

In [ ]:
# attack setup
tm = project.traceManager()
attack = cw.cpa(tm)
# set different than usual leakage model, as per http://wiki.newae.com/Tutorial_CW305-2_Breaking_AES_on_FPGA
leak_model = cw.AES128(cw.aes128leakage.LastroundStateDiff)
#leak_model = cw.AES128(cw.aes128leakage.SBox_output)
attack.setLeakModel(leak_model)
cb = cw.getJupyterCallback(attack)

In [ ]:
# run attack
attack_results = attack.processTraces(cb)

In [ ]:
# plot PGE
plot_data = cw.analyzerPlots(attack_results)
p = figure(plot_width=900)

for bnum in range(16):
    ret = plot_data.pgeVsTrace(bnum)
    p.line(ret[0], ret[1], line_color='red')
show(p)

In [ ]:
# Check correct key is received.
# Note that LastroundStateDiff model targets the round 10 key;
# we need to roll it back to compare against the unexpanded AES encryption key.
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import keyScheduleRounds
recv_lastroundkey = [kguess[0][0] for kguess in attack_results.findMaximums()]
recv_key = keyScheduleRounds(recv_lastroundkey, 10, 0)
key = list(key)
assert (key == recv_key), "Failed to recover encryption key\nGot:      {}\nExpected: {}".format(recv_key, key)